# Agenda

1. Chat Completions
2. Retrieval Augmented Generation
3. Integrating with Semantic Kernel
4. Evaluations + Benchmarking with Promptflow


# Hello There!

## Test

### Test 2

* This is a point
* This is another point
* This is yet another point

In [1]:
print("hello world")

hello world
